In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install jiwer
!pip install pandas
!pip install numpy


  Using cached jiwer-3.0.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached rapidfuzz-3.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached jiwer-3.0.4-py3-none-any.whl (21 kB)
Using cached rapidfuzz-3.9.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.4 MB)


In [3]:
from jiwer import wer, cer
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
#add on_bad_lines = skip to skip all line more than 8 values
amazon = pd.read_csv("/content/drive/MyDrive/DI_DP/amazon.csv", on_bad_lines='skip')
goodread = pd.read_csv("/content/drive/MyDrive/DI_DP/goodread.csv", on_bad_lines='skip')
wonderbk = pd.read_csv("/content/drive/MyDrive/DI_DP/wonderbk.csv", on_bad_lines='skip')

<ipython-input-4-de58e2a9a9a0>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  amazon = pd.read_csv("/content/drive/MyDrive/DI_DP/amazon.csv", on_bad_lines='skip')


In [5]:
amazon.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271279,271279,271277,271279,271277,271279,271279,271276
unique,271279,242056,101992,202,16803,270963,270963,270960
top,0767409752,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/0385722206.0...,http://images.amazon.com/images/P/0385722206.0...,http://images.amazon.com/images/P/068815333X.0...
freq,1,27,631,13899,7533,2,2,2


In [6]:
goodread.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11123.000000,11123.000000,1.112300e+04,11123.000000,1.112300e+04,11123.000000
mean,21310.856963,3.934075,9.759880e+12,336.405556,1.794285e+04,542.048099
std,13094.727252,0.350485,4.429758e+11,241.152626,1.124992e+05,2576.619589
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10277.500000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780582e+12,299.000000,7.450000e+02,47.000000
75%,32104.500000,4.140000,9.780872e+12,416.000000,5.000500e+03,238.000000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


In [7]:
wonderbk.describe()

,Title,Authors,Description,Category,Publisher,Publish Date,Price
count,103082,103082,70213,76912,103074,103082,103082
unique,97818,63580,68831,3106,13029,956,1387
top,The Velveteen Rabbit,By,For Ingest Only - Data needs to be cleaned up ...,"Fiction , General",Simon & Schuster,"Thursday, January 1, 2004",Price Starting at $5.29
freq,12,1043,30,2549,1521,868,41876


In [8]:
print(amazon.columns)
print(goodread.columns)
print(wonderbk.columns)

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')
Index(['Title', 'Authors', 'Description', 'Category', 'Publisher',
       'Publish Date', 'Price'],
      dtype='object')


**CHỈNH LẠI TÊN CÁC CỘT ĐỂ THỐNG NHẤT 1 KIỂU**

In [9]:
amazon.rename(columns={'ISBN': 'isbn', 'Book-Title': 'book_title', 'Book-Author': 'author', 'Year-Of-Publication': 'year', 'Publisher': 'publisher'}, inplace=True)
goodread.rename(columns={'bookID': 'book_id', 'title': 'book_title', 'authors': 'author', 'language_code': 'language'}, inplace=True)
wonderbk.rename(columns={'Title': 'title', 'Authors': 'author', 'Publisher': 'publisher', 'Publish Date': 'publication_date'}, inplace=True)

In [10]:
wonderbk.rename(columns={'Title': 'title', 'Authors': 'author', 'Publisher': 'publisher', 'Publish Date': 'publication_date'}, inplace=True)

In [11]:
print(amazon.columns)
print(goodread.columns)
print(wonderbk.columns)

Index(['isbn', 'book_title', 'author', 'year', 'publisher', 'Image-URL-S',
       'Image-URL-M', 'Image-URL-L'],
      dtype='object')
Index(['book_id', 'book_title', 'author', 'average_rating', 'isbn', 'isbn13',
       'language', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')
Index(['title', 'author', 'Description', 'Category', 'publisher',
       'publication_date', 'Price'],
      dtype='object')


**CHỈNH NGÀY THÁNG NĂM VỀ HẾT NĂM ĐỂ THỐNG NHẤT**

In [12]:
goodread['publication_date'][0]

'9/16/2006'

In [13]:
amazon['year'][0]

2002

In [14]:
wonderbk['publication_date'][1]

'Sunday, March 1, 1981'

In [16]:
for i in range(len(goodread)):
  goodread['publication_date'][i] = int(goodread['publication_date'][i].split("/")[-1])

Streaming output truncated to the last 5000 lines.
<ipython-input-16-ac8319e7dc26>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  goodread['publication_date'][i] = int(goodread['publication_date'][i].split("/")[-1])
<ipython-input-16-ac8319e7dc26>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  goodread['publication_date'][i] = int(goodread['publication_date'][i].split("/")[-1])
<ipython-input-16-ac8319e7dc26>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [18]:
for i in tqdm(range(len(wonderbk))):
    wonderbk['publication_date'][i] = int(wonderbk['publication_date'][i].split(", ")[-1].split()[-1])


100%|██████████| 103082/103082 [00:45<00:00, 2258.81it/s]


**CHỈNH LẠI TÊN CỘT PUBLIC DATE VỀ NĂM**

In [30]:
#Map to year
wonderbk.rename(columns={'publication_date': 'year'}, inplace=True)
goodread.rename(columns={'publication_date': 'year'}, inplace=True)
print(wonderbk.columns)
print(goodread.columns)

Index(['title', 'author', 'Description', 'Category', 'publisher', 'year',
       'Price'],
      dtype='object')
Index(['book_id', 'book_title', 'author', 'average_rating', 'isbn', 'isbn13',
       'language', '  num_pages', 'ratings_count', 'text_reviews_count',
       'year', 'publisher'],
      dtype='object')


NUMBER OF ELEMENT IN EACH CSV

In [20]:
print(len(amazon))
print(len(goodread))
print(len(wonderbk))

271279
11123
103082


**BỞI VÌ CÓ ISBN LÀ DUY NHẤT VỚI MỖI SÁCH NÊN MÌNH SẼ MERGE AMAZON VỚI GOODREAD TRƯỚC CHO DỄ, TẠO 1 MẢNG ĐỂ LƯU ISBN TRÙNG NHAU, SAU ĐÓ DUYỆT NẾU ISBN CỦA GOODREAD TRONG MẢNG THÌ KHÔNG THÊM VÀO => XONG**

In [21]:
goodread_isbn = list(goodread['isbn'])

In [22]:
map_amazon_goodread = []

In [23]:
for i in tqdm(range(len(amazon))):
  if amazon['isbn'][i] in goodread_isbn:
    id_goodread = goodread_isbn.index(amazon['isbn'][i])
    map_amazon_goodread.append(str(amazon['isbn'][i]))


100%|██████████| 271279/271279 [00:59<00:00, 4536.82it/s]


In [24]:
map_amazon_goodread[0]

'0440234743'

**MAP TỔNG CỘNG 5 TRƯỜNG: ISBN, BOOK TITLE, AUTHOR, PUBLISHER VÀ YEAR**

In [31]:
#we keep only 5 cols
col_merge = ['isbn', 'book_title', 'author', 'publisher', 'year']

In [32]:
ISBN = []
BOOK_TITLE = []
AUTHOR = []
PUBLISHER = []
YEAR = []
for i in tqdm(range(len(amazon))):
  ISBN.append(str(amazon['isbn'][i]))
  BOOK_TITLE.append(amazon['book_title'][i])
  AUTHOR.append(amazon['author'][i])
  PUBLISHER.append(amazon['publisher'][i])
  YEAR.append(str(amazon['year'][i]))
for i in tqdm(range(len(goodread))):
  if goodread['isbn'][i] in ISBN:
    continue
  else:
    ISBN.append(str(goodread['isbn'][i]))
    BOOK_TITLE.append(goodread['book_title'][i])
    AUTHOR.append(goodread['author'][i])
    PUBLISHER.append(goodread['publisher'][i])
    YEAR.append(str(goodread['year'][i]))

100%|██████████| 11123/11123 [00:44<00:00, 251.04it/s]


In [33]:
data = {
    'isbn': ISBN,
    'book_title': BOOK_TITLE,
    'author': AUTHOR,
    'publisher': PUBLISHER,
    'year': YEAR
}

# Create DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV
df.to_csv('/content/drive/MyDrive/DI_DP/amazon_goodread.csv')

**SAU KHI MAP, SỐ DỮ LIỆU CHÍNH BẰNG LEN(AMAZON + GOODREAD) - SỐ LƯỢNG ISBN TRÙNG: 278754, MÌNH CŨNG CÓ THỂ THẤY CÓ NHIỀU BOOK TITLE TRÙNG NHAU NHƯNG KHÁC SỐ ISBN**

In [36]:
df.describe()

,isbn,book_title,author,publisher,year
count,278754,278754,278752,278752,278754
unique,278754,247977,105603,17709,128
top,8497646983,Selected Poems,Agatha Christie,Harlequin,2002
freq,1,29,654,7537,18037


In [37]:
wonderbk.columns

Index(['title', 'author', 'Description', 'Category', 'publisher', 'year',
       'Price'],
      dtype='object')

In [51]:
wonderbk['author'][0]

'By Colton, Larry'

In [52]:
for i in tqdm(range(len(wonderbk))):
  #remove all by
  wonderbk['author'][i] = wonderbk['author'][i].replace("By ", "")

100%|██████████| 103082/103082 [00:59<00:00, 1746.87it/s]


In [54]:
for i in range(10):
  print(wonderbk['author'][i])

Colton, Larry
Grumbach, Doris
Piscatella, Joseph C.
Davis, Paul D.
Spangler, Amy
Short, Bo
Canfield, Jack (COM) and Hansen, Mark Victor (COM)
Stepanek, Mattie J. T.
Aksyonov, Vassily, Heim, Michael Henry, and Bouis, Antonina W.
Eakin, Katherine M. and Deaman, Joane (EDT)


In [46]:
cnt=0
for i in tqdm(range(len(df))):
  if "," in df['author'][i]:
    cnt+=1
    print(df['author'][i])
  if cnt==10:
    break

  2%|▏         | 4192/278754 [00:00<00:01, 162279.73it/s]

Scott, Cunningham
Mark E., Jr. Neely
Rush H., III Limbaugh
Wilson, III Simmons
Debra K., Dr. Shatoff
Eugene, Ph.D. Savaiano
Anthony, Md. Walker
Robert T., Ii Trotter
Landrum B., Md Shettles
Charles, Dr Stanley


**CHUẨN HÓA HẾT BOOK TITLE VỀ LOWER ĐỂ DỄ XỬ LÝ. CHECK NẾU CHARACTER ERROR RATE 2 BOOK TITLE <0.1 VÀ CHARACTER ERROR RATE CỦA AUTHOR SORT <0.2 THÌ GIỮ, DÙNG AUTHOR THẲNG KHÁ KHÓ KHĂN DO FORMAT CỦA WONDERBK VỚI AMAZON KHÁ LỘN XỘN => MÌNH SORT STRING**

In [58]:
for i in tqdm(range(len(df))):
  df['author'][i] = str(df['author'][i]).lower()
for i in tqdm(range(len(wonderbk))):
  wonderbk['author'][i] = str(wonderbk['author'][i]).lower()


100%|██████████| 103082/103082 [01:04<00:00, 1588.64it/s]


In [63]:
wonderbk.columns


Index(['title', 'author', 'Description', 'Category', 'publisher', 'year',
       'Price'],
      dtype='object')

In [64]:
df.columns

Index(['isbn', 'book_title', 'author', 'publisher', 'year'], dtype='object')

In [65]:
for i in tqdm(range(len(wonderbk))):
  wonderbk['title'][i] = str(wonderbk['title'][i]).lower()

100%|██████████| 103082/103082 [01:07<00:00, 1532.15it/s]


In [67]:
wonderbk.rename(columns={'title': 'book_title'}, inplace=True)


In [92]:
# Only need to run 1 time
# columns_to_drop = ['Description', 'Category', 'Price']
# wonderbk_drop = wonderbk.drop(columns=columns_to_drop)
# wonderbk_drop.to_csv("/content/drive/MyDrive/DI_DP/wonderbk_new.csv")


In [74]:
map_amazongoodread_to_wonderbk = []

In [78]:
!pip install pyspark python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 16.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=fbc7513878cf8d24eb254f0dd69947cebbead12e72891415442b148a42404803
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import Levenshtein

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CSV Merge by Similarity") \
    .getOrCreate()

csv_file1 = '/content/drive/MyDrive/DI_DP/amazon_goodread.csv'
csv_file2 = '/content/drive/MyDrive/DI_DP/wonderbk_new.csv'

df1 = spark.read.csv(csv_file1, header=True, inferSchema=True)
df2 = spark.read.csv(csv_file2, header=True, inferSchema=True)

print("change col")
df2 = df2.selectExpr(
    "author as author_2",
    "book_title as book_title_2",
    "publisher as publisher_2",
    "year as year_2"
)

# Define a UDF to calculate the character error rate
def CER(str1, str2):
    # if str1==str2:
    #   return 0
    if str1 and str2:
      return cer(str1, str2)
    else:
      return 1

def author_CER(str1, str2):
    # if str1==str2:
    #   return 0
    if str1 and str2:
      s1 = "".join(sorted(str(str1)))
      s2 = "".join(sorted(str(str2)))
      return cer(s1, s2)
    return 1

# print(author_CER("abc", "bc"))

cer_udf = udf(CER, FloatType())
cer_author_udf = udf(author_CER, FloatType())

# Perform a Cartesian join (cross join)
cross_joined_df = df1.crossJoin(df2)

# Calculate the CER for book_title and author
joined_with_cer = cross_joined_df.withColumn('title_cer', cer_udf(cross_joined_df['book_title'], cross_joined_df['book_title_2'])) \
                                 .withColumn('author_cer', cer_author_udf(cross_joined_df['author'], cross_joined_df['author_2']))
print("filter")
# Filter rows where CER for both book_title and author are below their respective thresholds
title_threshold = 0.1
author_threshold = 0.2
similar_titles_authors_df = joined_with_cer.filter((joined_with_cer['title_cer'] < title_threshold) &
                                                   (joined_with_cer['author_cer'] < author_threshold))

print("select")
# Select necessary columns to avoid duplicate columns from both DataFrames
# merged_df = similar_titles_authors_df.select(
#     'isbn',
#     'author',
#     'book_title',
#     'publisher',
#     'year',
#     'author_2',
#     'book_title_2',
#     'publisher_2',
#     'year_2'
# )

# # Show the result
# # merged_df.show()

# # Optionally, save the resulting DataFrame to a new CSV file
# merged_df.write.csv('/content/drive/MyDrive/DI_DP/merge.csv', header=True)

# # Stop the Spark session
# spark.stop()

change col
filter
select


In [1]:
# similar_titles_authors_df.count()
# df_repartitioned = similar_titles_authors_df.repartition(10)

# Now, count the rows (may be faster with fewer partitions)
# similar_titles_authors_df.first()

In [ ]:
first_row

**CHECK THỬ SỐ LƯỢNG AUTHOR VÀ TỔNG DỮ LIỆU**

In [60]:
print(len(list(set(list(wonderbk['author'])))))
print(len(list(set(list(df['author'])))))
print(len(wonderbk))
print(len(df))

63579
102902
103082
278754
